# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = "gpt-4o-mini"

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings

In [6]:
%pip install docarray

Note: you may need to restart the kernel to use updated packages.


In [7]:
embeddings = OpenAIEmbeddings()
# Create vector store
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model=llm_model)

response = index.query(query, llm=llm)

In [10]:
display(Markdown(response))

Here is a table listing the shirts with sun protection along with a summary of each:

| Name                                      | Description Summary                                                                                     |
|-------------------------------------------|--------------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt   | Lightweight, UPF 50+ sun protection, 100% polyester, wrinkle-resistant, with cape venting and pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve    | Designed for fishing, UPF 50+ coverage, 52% polyester and 48% nylon, moisture-wicking, with cape venting and pockets. |
| Men's TropicVibe Shirt, Short-Sleeve      | Lightweight, UPF 50+ rated, 71% nylon and 29% polyester, wrinkle-resistant, with cape venting and pockets. |
| Sun Shield Shirt                          | Slightly fitted, UPF 50+ rated, 78% nylon and 22% Lycra, moisture-wicking, abrasion resistant, handwash recommended. |

## The behind scences of index.query(query, llm=llm)

In [11]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [12]:
docs = loader.load()

In [13]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [14]:
embeddings = OpenAIEmbeddings()

In [15]:
embed = embeddings.embed_query("Hi my name is Joseph")

In [16]:
print(embed[:5])

[-0.03503866493701935, 0.010617966763675213, -0.017631957307457924, -0.013677593320608139, -0.02473980002105236]


In [17]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [18]:
query = "Please suggest a shirt with sunblocking"

In [19]:
docs = db.similarity_search(query)

In [20]:
len(docs)

4

In [21]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [22]:
retriever = db.as_retriever()

In [23]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [24]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [25]:
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [26]:
display(Markdown(response.content))

Here’s a summary of the shirts with sun protection in a markdown table format:

| Name                                   | Description Summary                                                                                                                                                                                                                     |
|----------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Sun Shield Shirt                       | High-performance sun shirt with UPF 50+ protection, blocking 98% of harmful UV rays. Slightly fitted, made of 78% nylon and 22% Lycra Xtra Life fiber. Moisture-wicking, abrasion-resistant, and comfortable over swimsuits. Handwash recommended. |
| Men's Plaid Tropic Shirt, Short-Sleeve| Lightweight, UPF 50+ rated shirt originally designed for fishing. Made of 52% polyester and 48% nylon, it features wrinkle-free fabric, evaporates perspiration quickly, and includes front and back cape venting with two bellows pockets.         |
| Men's TropicVibe Shirt, Short-Sleeve  | Lightweight sun-protection shirt with UPF 50+ rating. Traditional fit, made of 71% nylon and 29% polyester with a 100% polyester knit mesh lining. Features include wrinkle resistance, cape venting, and two front bellows pockets.                  |
| Men's Tropical Plaid Short-Sleeve Shirt| Lightest hot-weather shirt with UPF 50+ protection, made of 100% polyester. Traditional fit, wrinkle-resistant, with front and back cape venting and two front bellows pockets. Provides high sun protection by blocking 98% of harmful rays.         |

This table summarizes the key features and benefits of each shirt, focusing on their sun protection capabilities.

In [27]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [28]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [29]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [30]:
display(Markdown(response.get("result")))

Here is a table listing the shirts with sun protection along with a summary of each:

| Name                                      | Description Summary                                                                                     |
|-------------------------------------------|--------------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt   | Lightweight, UPF 50+ sun protection, 100% polyester, wrinkle-resistant, with cape venting and pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve    | Designed for fishing, UPF 50+ coverage, 52% polyester and 48% nylon, moisture-wicking, with cape venting and pockets. |
| Men's TropicVibe Shirt, Short-Sleeve      | Lightweight, UPF 50+ rated, 71% nylon and 29% polyester, wrinkle-resistant, with cape venting and pockets. |
| Sun Shield Shirt                          | Slightly fitted, UPF 50+ rated, 78% nylon and 22% Lycra, moisture-wicking, abrasion resistant, handwash recommended. |

In [31]:
response = index.query(query, llm=llm)
display(Markdown(response))

Here is a table listing the shirts with sun protection along with a summary of each:

| Name                                      | Description Summary                                                                                      |
|-------------------------------------------|---------------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt   | Lightweight, UPF 50+ sun protection, 100% polyester, wrinkle-resistant, with cape venting and bellows pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve    | Designed for fishing, UPF 50+ coverage, made of 52% polyester and 48% nylon, moisture-wicking, wrinkle-free, with cape venting and bellows pockets. |
| Men's TropicVibe Shirt, Short-Sleeve      | Lightweight, UPF 50+ rated, made of 71% nylon and 29% polyester, wrinkle-resistant, with cape venting and bellows pockets. |
| Sun Shield Shirt                          | Slightly fitted, UPF 50+ rated, made of 78% nylon and 22% Lycra Xtra Life, moisture-wicking, abrasion resistant, recommended by The Skin Cancer Foundation. |